In [54]:
import os

# Variables de entorno
os.environ['REDSHIFT_DBNAME'] = 'data-engineer-database'
os.environ['REDSHIFT_USER'] = 'jenni_salazar01_coderhouse'
os.environ['REDSHIFT_PASSWORD'] = 'N1u27vOhvm'
os.environ['REDSHIFT_HOST'] = 'data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com'
os.environ['REDSHIFT_PORT'] = '5439'

In [55]:
import os
import requests
import json
import psycopg2
from psycopg2 import sql
import pandas as pd

# Función para conectarse a la base de datos de Redshift
def conectar_redshift():
    print("Conectando a Redshift...")
    try:
        conn = psycopg2.connect(
            dbname=os.environ['REDSHIFT_DBNAME'],
            user=os.environ['REDSHIFT_USER'],
            password=os.environ['REDSHIFT_PASSWORD'],
            host=os.environ['REDSHIFT_HOST'],
            port=os.environ['REDSHIFT_PORT']
        )
        print("Conexión exitosa.")
        return conn
    except Exception as e:
        print("Error al conectar a Redshift:", e)
        return None

In [56]:
# Función para crear una tabla en Redshift
def crear_tabla_redshift(conn):
    cursor = conn.cursor()
    create_table_query = """
    CREATE TABLE IF NOT EXISTS data_criptomonedas (
        id VARCHAR(255),
        nombre VARCHAR(100),
        simbolo VARCHAR(10),
        precio FLOAT,
        market_cap_rank INTEGER,
        last_updated TIMESTAMP,
        current_price FLOAT,
        high_24h FLOAT,
        low_24h FLOAT,
        fecha_ingesta TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        PRIMARY KEY (id, last_updated)
    )
    """
    cursor.execute(create_table_query)
    conn.commit()
    cursor.close()

In [57]:
# Función para extraer datos de la API de CoinGecko
def extraer_datos_api():
    print("Extrayendo datos de la API de CoinGecko...")
    try:
        response = requests.get('https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd')
        data = response.json()
        print("Extracción exitosa.")
        return data
    except Exception as e:
        print("Error al extraer datos de la API:", e)
        return None

In [58]:
# Función para cargar los datos en Redshift
def cargar_datos_redshift(conn, data):
    if data:
        try:
            # Convertir los datos en un DataFrame
            df = pd.DataFrame(data)

            # Mostrar los primeros registros del DataFrame para verificar
            print("Datos extraídos:")
            print(df.head())

            # Carga en Redshift
            cursor = conn.cursor()
            try:
                # Vaciar la tabla antes de cargar nuevos datos
                cursor.execute("DELETE FROM data_criptomonedas")
                conn.commit()
                print("Tabla vaciada correctamente.")
            except Exception as e:
                print("Error al vaciar la tabla:", e)
                conn.rollback()
                return

            try:
                # Insertar datos en la tabla
                for index, row in df.iterrows():
                    cursor.execute("""
                        INSERT INTO data_criptomonedas
                        (id, nombre, simbolo, precio, market_cap_rank, last_updated, current_price, high_24h, low_24h)
                        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
                    """, (row['id'], row['name'], row['symbol'], row['current_price'], row['market_cap_rank'], row['last_updated'], row['current_price'], row['high_24h'], row['low_24h']))
                conn.commit()
                print("Datos cargados en Redshift correctamente.")
            except Exception as e:
                print("Error al cargar datos en Redshift:", e)
                conn.rollback()
                return
            finally:
                cursor.close()
        except Exception as e:
            print("Error al convertir los datos en DataFrame:", e)
    else:
        print("No se pudieron extraer datos de la API.")


In [59]:
# Ejecutar el script
conn = conectar_redshift()
if conn:
    data = extraer_datos_api()
    if data:
        cargar_datos_redshift(conn, data)
    else:
        print("No se pudieron extraer datos de la API.")
else:
    print("No se pudo conectar a Redshift.")

Conectando a Redshift...
Conexión exitosa.
Extrayendo datos de la API de CoinGecko...
Extracción exitosa.
Datos extraídos:
            id symbol      name  \
0      bitcoin    btc   Bitcoin   
1     ethereum    eth  Ethereum   
2       tether   usdt    Tether   
3  binancecoin    bnb       BNB   
4       solana    sol    Solana   

                                               image  current_price  \
0  https://assets.coingecko.com/coins/images/1/la...      64693.000   
1  https://assets.coingecko.com/coins/images/279/...       3101.060   
2  https://assets.coingecko.com/coins/images/325/...          1.001   
3  https://assets.coingecko.com/coins/images/825/...        565.160   
4  https://assets.coingecko.com/coins/images/4128...        147.280   

      market_cap  market_cap_rank  fully_diluted_valuation  total_volume  \
0  1274639506007                1             1.359597e+12   26287063357   
1   372566031165                2             3.725660e+11    9506650158   
2   1096639